In [16]:
# IMPORTAÇÕES
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [17]:
# Carregar dataset
df = pd.read_csv("fiis.csv", sep=';', encoding='latin-1')

In [18]:
df

,Papel,Segmento,Cotação,FFO Yield,Dividend Yield,P/VP,Valor de Mercado,Liquidez,Qtd de imóveis,Preço do m2,Aluguel por m2,Cap Rate,Vacância Média
0,AAZQ11,Títulos e Val. Mob.,"7,42","14,48%","16,00%","0,86",178357000,610697,0,0,0,"0,00%","0,00%"
1,ABCP11,Shoppings,"84,04","8,30%","7,98%","0,78",395751000,68335,1,"5517,42","567,85","10,29%","2,90%"
2,AFHI11,Títulos e Val. Mob.,"92,65","11,60%","12,27%","0,98",422078000,725888,0,0,0,"0,00%","0,00%"
3,AGRX11,Híbrido,"7,78","21,74%","15,59%","0,76",139287000,239179,0,0,0,"0,00%","0,00%"
4,AIEC11,Lajes Corporativas,"45,25","17,99%","15,75%","0,58",218331000,285262,2,"8785,5","2375,61","27,04%","0,00%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,XPSF11,Títulos e Val. Mob.,"6,3","13,34%","12,74%","0,82",272803000,415496,0,0,0,"0,00%","0,00%"
372,ZAGH11,Híbrido,"10,39","5,24%","7,33%","1,11",92384400,13806,1,"2562,99",81,"3,16%","0,00%"
373,ZAVC11,Títulos e Val. Mob.,"8,7","15,10%","17,55%","0,87",26946900,26128,0,0,0,"0,00%","0,00%"
374,ZAVI11,Híbrido,"93,58","16,70%","15,29%","0,72",109799000,75795,9,"1509,96","223,78","14,82%","0,83%"


In [19]:
# NORMALIZAÇÃO
colunas_numericas = [
    'Cotação', 'FFO Yield', 'Dividend Yield', 'P/VP', 'Valor de Mercado',
    'Liquidez', 'Qtd de imóveis', 'Preço do m2', 'Aluguel por m2',
    'Cap Rate', 'Vacância Média'
]

for col in colunas_numericas:
    df[col] = df[col].astype(str).str.replace(',', '.').str.replace('%', '')
    df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.dropna().copy()

In [28]:
# TRIAGEM
# Defina aqui o valor maximo da cotação
cotacao = 10

# Defina aqui o valor mínimo de Dividend Yield
dy_minimo = 8

# Defina aqui o valor máximo para o P/VP
pvp_teto_risco = 5

# Defina aqui o valor da liquidez mínima
liquidez_minima = 100000

# Defina aqa quantidade mínima de imóveis
qtd_imoveis_minima = 3



In [29]:
# Realiza filtragem
df = df[df['Cotação'] <= cotacao].copy()
df = df[df['Dividend Yield'] >= dy_minimo].copy()
df = df[df['P/VP'] <= pvp_teto_risco].copy()
df = df[df['Liquidez'] >= liquidez_minima].copy()
df = df[df['Qtd de imóveis'] >= qtd_imoveis_minima].copy()

# Exibe o resultado
# print(f"\nFIIs com Dividend Yield >= {dy_minimo}%: {len(df)}")
print(f"\nFIIs Restantes: {len(df)}")


FIIs Restantes: 8


In [30]:
# Indicadores, valores ideais e pesos
metricas = {
    'Dividend Yield': {'ideal': 8.0, 'peso': 3, 'tipo': 1},
    'P/VP': {'ideal': 1.0, 'peso': 2, 'tipo': 0},
    'Cap Rate': {'ideal': 8.0, 'peso': 2, 'tipo': 1},
    'Vacância Média': {'ideal': 5.0, 'peso': 3, 'tipo': 0},
    'Liquidez': {'ideal': 200000, 'peso': 1, 'tipo': 1},
    'FFO Yield': {'ideal': 8.0, 'peso': 1, 'tipo': 1},
    'Cotação': {'ideal': 10.0, 'peso': 3, 'tipo': 0}
}

In [31]:
scores = []
for idx, row in df.iterrows():
    score = 0
    total_peso = 0
    for col, config in metricas.items():
        ideal = config['ideal']
        peso = config['peso']
        tipo = config['tipo']
        valor = row[col]

        # Evita divisões por 0
        if pd.isnull(valor) or valor == 0:
            continue

        # Cálculo do desvio relativo
        if tipo == 1:
            desvio = max(0, (ideal - valor) / ideal)
        elif tipo == 0:
            desvio = max(0, (valor - ideal) / ideal)

        score += desvio * peso
        total_peso += peso

    final_score = score / total_peso if total_peso else 999
    scores.append(final_score)

df['Score'] = scores

In [32]:
# Ranking final (menor score = mais próximo do ideal)
ranking = df.sort_values(by='Score', ascending=True).reset_index(drop=True)

# Exibe os 10 melhores
ranking[['Papel', 'Cotação', 'Segmento', 'Dividend Yield', 'P/VP', 'Cap Rate', 'Vacância Média', 'Liquidez', 'FFO Yield', 'Score']]

,Papel,Cotação,Segmento,Dividend Yield,P/VP,Cap Rate,Vacância Média,Liquidez,FFO Yield,Score
0,AJFI11,7.42,Títulos e Val. Mob.,10.90,0.61,12.05,3.30,772855,11.17,0.000000
1,ALZR11,9.99,Híbrido,9.24,0.96,8.20,0.00,2471870,8.38,0.000000
2,VIUR11,5.75,Outros,13.81,0.66,14.63,3.14,239801,14.31,0.000000
3,SNAG11,9.68,Híbrido,12.77,0.95,9.65,0.00,1366820,11.46,0.000000
4,VRTM11,7.11,Híbrido,15.21,0.76,0.00,0.00,435221,10.05,0.000000
5,CPSH11,9.70,Shoppings,12.34,0.82,7.46,0.00,2482100,5.82,0.033958
6,VINO11,5.08,Lajes Corporativas,12.12,0.49,16.61,7.82,616572,10.75,0.112800
7,RBVA11,8.70,Outros,11.78,0.08,110.74,8.76,1864720,84.38,0.150400
